<a href="https://colab.research.google.com/github/leonardoLavagna/Drug-repurposing/blob/main/BI_LeoL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import math
import json
import os

## 1) Get the data

In [ ]:
# read the human interactome data

homo_sapiens_network = pd.read_csv("/content/Data/BIOGRID-ORGANISM-Homo_sapiens-4.4.204.tab3.txt", sep = "\t")

Filter out the same organism A and B:

In [ ]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Organism Name Interactor A'] == homo_sapiens_network['Organism Name Interactor B']]

Filter out the same Gene Interactor A and B (self-loops):

In [ ]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Official Symbol Interactor A'] != homo_sapiens_network['Official Symbol Interactor B']]

Filter out only the physical interactions:

In [ ]:
homo_sapiens_network = homo_sapiens_network[homo_sapiens_network['Experimental System Type'] == "physical"]

To avoid duplicating edges A-B and B-A, we'll filter them in few steps later... Now we focus on the curated disease-gene association

In [ ]:
gene_dis_association = pd.read_csv('/content/drive/MyDrive/BI - Data/curated_gene_disease_associations.tsv', sep = '\t')
gene_dis_association = gene_dis_association[gene_dis_association['diseaseName'] == '..........'] 

In [ ]:
gene_dis_association

Effective number of disease genes present in the homo sapiens network:

In [ ]:
disease_genes_in_interactome = set.intersection(set(gene_dis_association['geneSymbol'].tolist()), set(homo_sapiens_network['Official Symbol Interactor A'].tolist()).union(homo_sapiens_network['Official Symbol Interactor B'].tolist()))

### Create human interactome

In [ ]:
human_interactome = nx.Graph()

In [ ]:
human_interactome.add_nodes_from(map(int, list(set(homo_sapiens_network["Entrez Gene Interactor A"].tolist() + homo_sapiens_network["Entrez Gene Interactor B"].tolist())))) # considering the Gene IDs
# human_interactome.add_nodes_from(list(set(homo_sapiens_network["Official Symbol Interactor A"].tolist() + homo_sapiens_network["Official Symbol Interactor B"].tolist())))

Now we filter the duplicates edges A-B, B-A

In [ ]:
human_interactome.add_edges_from(list(zip(map(int, homo_sapiens_network["Entrez Gene Interactor A"]), map(int, homo_sapiens_network["Entrez Gene Interactor B"]))))
# human_interactome.add_edges_from(zip(homo_sapiens_network["Official Symbol Interactor A"], homo_sapiens_network["Official Symbol Interactor B"]))

### LLC

In [ ]:
LCCs = [len(human_interactome) for human_interactome in sorted(nx.connected_components(human_interactome), key=len, reverse=True)]
print(f"The largest component {LCCs[0]}") 

In [ ]:
print(LCCs)

In [ ]:
# extract first LLC
human_interactome_LCC = human_interactome.subgraph(max(nx.connected_components(human_interactome), key=len))
print(len(human_interactome_LCC)) # number of nodes in the largest component
print(len(human_interactome_LCC.edges()))

In [ ]:
# save graph
nx.write_adjlist(human_interactome_LCC, "test.adjlist")